In [1]:
from ActivationStoreParallel import ActivationsStore
from sparse_transcoder import SparseTranscoder
from transcoder_training_parallel import train_transcoder_on_language_model_parallel
from transcoder_runner_parallel import language_model_transcoder_runner_parallel
from dataclasses import dataclass
import transformer_lens
import torch
import wandb
from typing import Optional

In [4]:
@dataclass
class UnifiedConfig():
    # Common settings
    model_name: str = "gpt2-small"
    hook_point: str = "blocks.10.hook_resid_pre"
    ln: str = 'blocks.10.ln1.hook_scale'
    hook_point_layer: int = 10
    layer: int = 10
    d_in: int = 768
    d_out: int = 768
    n_head: int = 12
    d_head: int = 64
    dataset_path: str = "Skylion007/openwebtext"
    is_dataset_tokenized: bool = False
    training: bool = True
    attn_scores_normed = False
    
    # SAE Parameters
    expansion_factor: int = 12   # TODO: NOT being used??
    d_hidden: int = 2400
    b_dec_init_method: str = "mean"
    
    # Training Parameters
    lr: float = 1e-5
    reg_coefficient: float = 4e-6
    lr_scheduler_name: Optional[str] = None
    train_batch_size: int = 2048
    context_size: int = 256
    lr_warm_up_steps: int = 5000
    
    # Activation Store Parameters
    n_batches_in_buffer: int = 128
    total_training_tokens: int = 20_000 * 10
    store_batch_size: int = 32
    use_cached_activations: bool = False
    
    # Resampling protocol
    feature_sampling_method: str = 'none'
    feature_sampling_window: int = 1000
    feature_reinit_scale: float = 0.2
    resample_batches: int = 1028
    dead_feature_window: int = 50000
    dead_feature_threshold: float = 1e-6
    
    # WANDB
    log_to_wandb: bool = False
    wandb_project: str = "sparsification"
    wandb_entity: Optional[str] = None
    wandb_log_frequency: int = 1000
    entity: str = "kwyn390"
    
    # Misc
    device: str = "cuda"
    eps: float = 1e-7
    seed: int = 42
    reshape_from_heads: bool = True
    n_checkpoints: int = 10
    checkpoint_path: str = "checkpoints"
    dtype: torch.dtype = torch.float32
    run_name: str = "qk_parallel"
    
    # Query-specific settings
    hook_transcoder_in_q: str = "blocks.10.hook_resid_pre"
    hook_transcoder_out_q: str = "blocks.10.attn.hook_q"
    target_q: str = "blocks.10.attn.hook_q"
    type_q: str = "resid_to_queries"
    
    # Key-specific settings
    hook_transcoder_in_k: str = "blocks.10.hook_resid_pre"
    hook_transcoder_out_k: str = "blocks.10.attn.hook_k"
    target_k: str = "blocks.10.attn.hook_k"
    type_k: str = "resid_to_keys"

cfg = UnifiedConfig()
cfg.run_name = f"{cfg.d_hidden}_{cfg.reg_coefficient}_{cfg.lr}"

In [5]:
sparse_transcoder_Q, sparse_transcoder_K = language_model_transcoder_runner_parallel(cfg)

285| MSE Loss 22.481:   0%|          | 585728/200000000 [00:50<3:02:51, 18175.88it/s]

Loaded pretrained model gpt2-small into HookedTransformer
Dataset is not tokenized! Updating config.
TRAIN STARTED
gonna schedule!
Reinitializing b_dec with mean of activations
Previous distances: 27.712806701660156
New distances: 19.674325942993164
Reinitializing b_dec_out with mean of activations
Previous distances: 36.17184066772461
New distances: 28.02600860595703
Reinitializing b_dec with mean of activations
Previous distances: 27.712806701660156
New distances: 19.674325942993164
Reinitializing b_dec_out with mean of activations
Previous distances: 41.437225341796875
New distances: 31.178068161010742
gonna progress bar!



Training SAE:   0%|          | 0/200000 [00:00<?, ?it/s]
0| MSE Loss 132.460:   0%|          | 0/200000 [00:00<?, ?it/s]
1| MSE Loss 129.409:   1%|          | 2048/200000 [00:00<00:11, 17822.29it/s]
1| MSE Loss 129.409:   2%|▏         | 4096/200000 [00:00<00:05, 35352.00it/s]
2| MSE Loss 122.982:   2%|▏         | 4096/200000 [00:00<00:05, 35352.00it/s]
3| MSE Loss 116.592:   3%|▎         | 6144/200000 [00:00<00:05, 35352.00it/s]
3| MSE Loss 116.592:   4%|▍         | 8192/200000 [00:00<00:08, 22629.59it/s]
4| MSE Loss 116.331:   4%|▍         | 8192/200000 [00:00<00:08, 22629.59it/s]
5| MSE Loss 109.755:   5%|▌         | 10240/200000 [00:00<00:08, 22629.59it/s]
5| MSE Loss 109.755:   6%|▌         | 12288/200000 [00:00<00:09, 20646.22it/s]
6| MSE Loss 109.612:   6%|▌         | 12288/200000 [00:00<00:09, 20646.22it/s]
7| MSE Loss 101.517:   7%|▋         | 14336/200000 [00:00<00:08, 20646.22it/s]
7| MSE Loss 101.517:   8%|▊         | 16384/200000 [00:00<00:08, 20964.06it/s]
8| MSE Loss 97.

Saved model to checkpoints/20480_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt
Saved model to checkpoints/20480_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt



12| MSE Loss 85.221:  12%|█▏        | 24576/200000 [00:01<00:08, 20860.55it/s]
13| MSE Loss 82.023:  13%|█▎        | 26624/200000 [00:01<00:08, 20860.55it/s]
13| MSE Loss 82.023:  14%|█▍        | 28672/200000 [00:01<00:07, 21527.46it/s]
14| MSE Loss 78.346:  14%|█▍        | 28672/200000 [00:01<00:07, 21527.46it/s]
15| MSE Loss 75.706:  15%|█▌        | 30720/200000 [00:01<00:07, 21527.46it/s]
15| MSE Loss 75.706:  16%|█▋        | 32768/200000 [00:01<00:07, 21100.59it/s]
16| MSE Loss 72.907:  16%|█▋        | 32768/200000 [00:01<00:07, 21100.59it/s]
17| MSE Loss 69.024:  17%|█▋        | 34816/200000 [00:01<00:07, 21100.59it/s]
17| MSE Loss 69.024:  18%|█▊        | 36864/200000 [00:01<00:06, 23353.52it/s]
18| MSE Loss 67.870:  18%|█▊        | 36864/200000 [00:01<00:06, 23353.52it/s]
19| MSE Loss 64.194:  19%|█▉        | 38912/200000 [00:01<00:06, 23353.52it/s]
19| MSE Loss 64.194:  20%|██        | 40960/200000 [00:01<00:07, 22181.89it/s]
20| MSE Loss 65.163:  20%|██        | 40960/200000 

...

Saved model to checkpoints/161792_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt
Saved model to checkpoints/161792_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt



82| MSE Loss 30.866:  84%|████████▍ | 167936/200000 [00:09<00:01, 16037.84it/s]
82| MSE Loss 30.866:  85%|████████▍ | 169984/200000 [00:09<00:01, 19086.35it/s]
83| MSE Loss 30.952:  85%|████████▍ | 169984/200000 [00:10<00:01, 19086.35it/s]
83| MSE Loss 30.952:  86%|████████▌ | 172032/200000 [00:10<00:01, 15849.28it/s]
84| MSE Loss 34.458:  86%|████████▌ | 172032/200000 [00:10<00:01, 15849.28it/s]
85| MSE Loss 29.644:  87%|████████▋ | 174080/200000 [00:10<00:01, 15849.28it/s]
86| MSE Loss 30.522:  88%|████████▊ | 176128/200000 [00:10<00:01, 15849.28it/s]
86| MSE Loss 30.522:  89%|████████▉ | 178176/200000 [00:10<00:00, 21947.55it/s]
87| MSE Loss 32.424:  89%|████████▉ | 178176/200000 [00:10<00:00, 21947.55it/s]
88| MSE Loss 33.976:  90%|█████████ | 180224/200000 [00:10<00:00, 21947.55it/s]
88| MSE Loss 33.976:  91%|█████████ | 182272/200000 [00:10<00:00, 20826.84it/s]
89| MSE Loss 33.516:  91%|█████████ | 182272/200000 [00:10<00:00, 20826.84it/s]
90| MSE Loss 31.006:  92%|█████████▏| 1

Saved model to checkpoints/180224_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt
Saved model to checkpoints/180224_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt



92| MSE Loss 29.061:  94%|█████████▍| 188416/200000 [00:10<00:00, 24291.27it/s]
93| MSE Loss 31.397:  95%|█████████▌| 190464/200000 [00:10<00:00, 24291.27it/s]
93| MSE Loss 31.397:  96%|█████████▋| 192512/200000 [00:10<00:00, 27273.01it/s]
94| MSE Loss 31.165:  96%|█████████▋| 192512/200000 [00:11<00:00, 27273.01it/s]
95| MSE Loss 34.666:  97%|█████████▋| 194560/200000 [00:11<00:00, 27273.01it/s]
95| MSE Loss 34.666:  98%|█████████▊| 196608/200000 [00:11<00:00, 21734.33it/s]
96| MSE Loss 29.405:  98%|█████████▊| 196608/200000 [00:11<00:00, 21734.33it/s]
97| MSE Loss 30.004:  99%|█████████▉| 198656/200000 [00:11<00:00, 21734.33it/s]
97| MSE Loss 30.004: : 200704it [00:11, 17817.05it/s]                          


Saved model to checkpoints/final_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt
Saved model to checkpoints/final_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt


In [7]:
sparse_transcoder_K.get_name()

'sparse_transcoder_gpt2-small_resid_to_keys_2400'

In [2]:
kl_loss = torch.nn.KLDivLoss(reduction="batchmean", log_target = True)


In [54]:
a = torch.rand(12, 256, 256)
b = torch.rand(12, 256, 256)


a_patt = a.softmax(-1)
b_patt = b.softmax(-1)

print(kl_loss(a_patt, b_patt).mean())

tensor(0.0819)


In [ ]:
kl_loss = torch.nn.KLDivLoss(reduction="batchmean", log_target = True)

kl_loss(a_patt, b_patt)